In [2]:
import pymc3 as pm
import numpy as np
import theano

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Discussion here: https://discourse.pymc.io/t/compounds-steps/954

In brief, when Compound steps are involved, it takes a list of `step` to generate a list of `methods`. So for example if you do
```python
with pm.Model() as m:
    rv1 = ...
    ...
    step1 = pm.Metropolis([rv1, rv2])
    step2 = pm.CategoricalGibbsMetropolis([rv3])
    trace = pm.sample(..., step=[step1, step2]...)
```
The Compound step is now contain a list of `methods`:
https://github.com/pymc-devs/pymc3/blob/999661c092310b1f247f14037f795a852425e9c9/pymc3/step_methods/compound.py#L22-L32

And at each sample, it iterates each method, which takes a `point` as input, and generates a new `point` as output. The new `point` is proposed within each step via a stochastic kernel, and if the proposal was rejected by MH criteria it just outputs the original input `point`

Take a simple example:

In [3]:
n_=theano.shared(np.asarray([10, 15]))
with pm.Model() as m:
    p = pm.Beta('p', 1., 1.)
    ni = pm.Bernoulli('ni', .5)
    k = pm.Binomial('k', p=p, n=n_[ni], observed=4)
#     trace = pm.sample()

There are two free parameters in the model we would like to sample from:

In [4]:
m.free_RVs

[p_logodds__, ni]

In [19]:
with pm.Model() as m:
    a = pm.Normal('a', 0., 1., shape=(3, 2))
    b = pm.Normal('b', shape=(1, 3))
#     step = pm.Metropolis([a, b])
#     step2 = pm.Metropolis([b])
#     pm.sample(step=[step, step2])

In [20]:
m.free_RVs

[a, b]

In [17]:
with pm.Model():
    a = pm.Normal('a', 0., 1., shape=(3, 2))
    b = pm.Normal('b', shape=(1, 3))
    step = pm.Metropolis([a, b], blocked=True)
    pm.sample(step=step)

Multiprocess sampling (2 chains in 2 jobs)
Metropolis: [b, a]
100%|██████████| 1000/1000 [00:01<00:00, 784.82it/s]
The gelman-rubin statistic is larger than 1.2 for some parameters.
The estimated number of effective samples is smaller than 200 for some parameters.


When we call `pm.sample()`, `PyMC3` assigns the best step method to each of them. For example, NUTS was assigned to `p_logodds__` and BinaryGibbsMetropolis was assigned to `ni`.

But we can also specify the step manually:

In [5]:
with m:
    step1 = pm.Metropolis([p])
    step2 = pm.BinaryGibbsMetropolis([ni])
    pm.sample(step=[step1, step2])

And now you can pass a point to the step, and see what happens:

In [6]:
point = m.test_point
point

{'ni': array(0), 'p_logodds__': array(0.)}

In [11]:
point, state = step1.step(point=point)
point, state

({'ni': array(0), 'p_logodds__': array(-0.06105001)},
 [{'accept': 1.0570269695563923, 'tune': True}])

In [12]:
step1.astep??

as you can see, the value of  `ni` does not change, but `p_logodds__` is updated.

And similarly, you can get a sample using the `step2`:

(notice that there is no `generates_stats`, so only output the point here.)

In [13]:
point = step2.step(point=point)
point

{'ni': array(1), 'p_logodds__': array(-0.06105001)}

In [ ]:
step2.

Compound step works exactly like this by iterating all the steps within the list. In effect, it is a metropolis hastings within gibbs sampling. 

Moreover, `pm.CompoundStep` is called internally by `pm.sample()`. We can make them explicit as below:

In [14]:
with m:
    comp_step1 = pm.CompoundStep([step1, step2])
comp_step1.methods

In [ ]:
with m:
    pm.sample(step=comp_step1)

Notes that, when in the default setting, the parameter update order follows the same order of the RVs, and it is assigned automatically. But if you specify the step you can change that order as well:

In [15]:
with m:
    comp_step2 = pm.CompoundStep([step2, step1])
comp_step2.methods

In the sampling, it always follows the same order per sample in the Gibbs like update. Notice that it is not exactly Gibbs sampling as it does not generate from a conditional probability. More precisely it updates in a Gibbs like fashion where the accept-reject is based on comparing the ratio of the conditional logp with $p \sim \text{Uniform}(0., 1.)$

A recurrent issue/concern is the validately of mixing discrete and continuous sampling, especially mixing other sampler with NUTS. While in BDA 3rd edition Chapter 12.4 there is a small paragraph on this "Combining HMC with Gibbs sampling", which suggesting this could be a valid way to do, the Stan devs always skeptical about how practical it is ([1](http://discourse.mc-stan.org/t/mcmc-sampling-does-not-work-when-execute/1918/47), [2](http://discourse.mc-stan.org/t/constraining-latent-factor-model-baysian-probabalisic-matrix-factorization-to-remove-multimodality/2152/21)). 


The concern with mixing discrete and continuous sampling is that the change in discrete parameters will affect the continuous distribution's geometry so that the adaptation (i.e., the tuned mass matrix and step size) may be inappropriate. HMC/NUTS is hypersensitive to its tuning parameters (mass matrix and step size). We also don't know how many iterations we have to run to get a decent sample when the discrete parameters change. We haven't evaluated any of this, in my experience if the discrete parameter is low dimension (e.g., 2 class mixture equivalent, outliner detection with explicit discrete labelling) it works OK. However, it is much less efficient than marginalize out the discrete parameters, and you do see the chain being stuck quite often.
It would be good to evaluate this more properly, eg to simulate data and look at posterior coverage, as explained in [Cook, Gelman, and Rubin 2006](https://amstat.tandfonline.com/doi/abs/10.1198/106186006x136976).